In [12]:
#!pip install git+https://github.com/diannamcallister/datefinder

url = "https://arabic.rt.com/features/1045499-%D8%AA%D8%B1%D9%85%D9%8A%D9%85-%D9%82%D8%B5%D8%B1-%D9%85%D8%AD%D9%85%D8%AF-%D8%B9%D9%84%D9%8A-%D8%A7%D9%84%D8%AA%D8%A7%D8%B1%D9%8A%D8%AE%D9%8A-%D9%81%D9%8A-%D9%85%D8%B5%D8%B1/"

In [13]:
import sqlite3 as lite
import re
import datefinder
import numpy as np
from urllib import request

con = lite.connect("googledata.sqlite")
cur = con.cursor()
cur.execute("""CREATE TABLE  IF NOT EXISTS articles(
                    id INTEGER PRIMARY KEY, title TEXT, article TEXT , date DATETIME)""")


html = request.urlopen(url).read().decode('utf8')
html[:60]

from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
title = soup.find('title')

#print(title) # Prints the tag
output_title = title.string # Prints the tag string content
#html_page = html.content
#soup = BeautifulSoup(html_page, 'html.parser')
text = soup.find_all(text=True)

output = ''
blacklist = [
    '[document]',
    'noscript',
    'header',
    'html',
    'meta',
    'head', 
    'input',
    'script',
    'img']

for t in text:
    if t.parent.name not in blacklist:
        output += '{} '.format(t)

regex_for_removing_english = re.compile('[a-zA-Z0-9\!@#$\%\^\&\*\(\)\,\.\/\?\"\:\{\}\|;\-<>\[\]]')
article = regex_for_removing_english.sub('', output)
article_list = article.splitlines()
final_art = [i for i in article_list if len(i) > 80]
final_art = ' \n '.join(final_art)

try:
    finaldate = ''
    date_matches = datefinder.find_dates(html)
    final_dates = []
    for match in date_matches:
        final_dates.append(str(match))
    len_dates = [len(i) for i in final_dates]
    finaldate = final_dates[np.argmax(len_dates)]
except:
    print('exceptionhappened')
    finaldate = ''

date_matches = datefinder.find_dates(html)
for match in date_matches:
    final_date = str(match)
cur.execute("""INSERT INTO articles ( title, article , date ) VALUES(?,?,?) """,(output_title,final_art, finaldate ) )

con.commit()


In [14]:
output_title

'ترميم قصر محمد علي التاريخي في مصر - RT Arabic'

In [15]:
final_art

'  РУС       ИНОТВ Д   \xa0      مباشر \xa0 الأخبار العالم العربي أخبار العالم روسيا الاقتصاد الرياضة البرامج بالتفاصيل قصارى القول اسأل أكثر رحلة في الذاكرة بانوراما فن الحياة أفلام وثائقية تحقيقات  منوعات العلوم والتكنولوجيا الصحة الثقافة والفن الفضاء العالم الرقمي عالم السيارات متفرقات مجتمع ميديا فيديوهات تقارير مصورة إنفوجرافيك جولات مصورة   صحافة ورأي   فيديوهات  инотв д     \n                      بوتين العمل على تشكيل اللجنة الدستورية السورية على وشك الانتهاء  \n             تتواصل في مصر أعمال ترميم القصر التاريخي لمؤسس مصر الحديثة محمد علي، ويضم القصر عددا من المباني المغلقة منذ عام  \n  زاهي حواس يعلن عن موعد كشف سبب وفاة الملك توت عنخ آمون مصر تعلن عن طفرة في حجم التجارة الخارجية صور الجيش المصري ينقذ سياحا تعطل يختهم في البحر الأحمر في قلب القاهرة ماسح أحذية يعمل وسيطا في بيع الأعضاء البشرية    \n \t\t\t\t\tمفوضة حقوق الأطفال في روسيا حماية الأطفال رسالة تساعد على المصالحة بين الجميع في سوريا \n                         البيت الأبيض ينشر صورا تظهر آثار الهجوم على معملي أرام

In [16]:
finaldate

'2019-09-16 11:07:39+00:00'